**qAIntum.ai**

# **Quantum Neural Network (QNN) Classifier**

This note is an example of using Photonic Analog (PA) QNN for binary classification. This is an application of the original work ["Continuous Variable Quantum Neural Networks"](https://arxiv.org/abs/1806.06871).

Compared to Classical Neural Networks, PA QNNs have a reduced number of parameters to train and converge faster with fewer epochs. However, this is a quantum algorithm simulated on classical computers hence the training time for quantum circuits tend to be longer than classical models.

The dataset used in this example can be found at https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

This file is organized in the following order:
0. Install and import necessary packages
1. Load and preprocess data (The data from Kaggle is saved as 'financial.csv'.)
2. Data encoding
3. QNN model
   * QNN layer
   * QNN circuit
   * Model building
4. Model training
5. Evaluation

For the open source repository, refer to https://github.com/qaintumai/quantum.

## **0. Intall Packages**

Ensure you have installed the packages detailed in requirements.txt before running the code below.

[Pennylane](https://pennylane.ai/) is a Python based quantum machine learning library by Xanadu. An old version is necessary for this example: v0.29.1.

In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml

## **1. Load and Preprocess Data**

Data: 

Data points: 

Number of features: 

Label: 

For the purpose of this experiment, we are using only ___ data points for training and __ for testing.

data source: 

In [18]:
#LOADING DATA
import sys
import os
script_dir = os.getcwd()
data_path = os.path.join(script_dir, '../examples/data/pima-indians-diabetes.csv')
data_path = os.path.normpath(data_path)

# Load the dataset
dataset = np.loadtxt(data_path, delimiter=',')

print("length of dataset: ", len(dataset))

X = dataset[:,0:8]
y = dataset[:,8]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

print(f"Size of X tensor: {X.size()} and first element of X: {X[0]}")
print(f"Size of Y tensor: {y.size()} and first element of Y: {y[0]}")

length of dataset:  4
Size of X tensor: torch.Size([4, 8]) and first element of X: tensor([  0.0000, 137.0000,  40.0000,  35.0000, 168.0000,  43.1000,   2.2880,
         33.0000])
Size of Y tensor: torch.Size([4, 1]) and first element of Y: tensor([1.])


## **2. Data Encoding**

This step converts classical data into a quantum state by using the data entries as parameters of the quantum gates.

The data encoding gates used are Squeezing, Rotation, Beamsplitter, Displacement Gate, and Kerr Gate. Other gates under "CV operators" in the Pennylane package can be explored.

In [19]:
def encode(x, num_wires):
        """
        Encodes the input data into a quantum state to be operated on using a sequence of quantum gates.

        Parameters:
        x : input data (list or array-like)

        The encoding process uses the following gates in sequence:
        - Squeezing gates: 2*self.num_wires parameters
        - Beamsplitter gates: 2(self.num_wires-1) parameters
        - Rotation gates: self.num_wires parameters
        - Displacement gates: 2*self.num_wires parameters
        - Kerr gates: self.num_wires parameters
          Total: 8*self.num_wires - 2 parameters

        rounds: the number of iterations of the sequence needed to take in all the entries of the input data
                num_features // (8 * self.num_wires - 2)
                We are adding (8 * self.num_wires - 3) as a pad to run one extra round for the remainding data entries.
        """
        num_features = len(x)
        

        # Calculate the number of rounds needed to process all features
        rounds = (num_features + (8 * num_wires - 3)) // (8 * num_wires - 2)
        print("encoding rounds", rounds)

        for j in range(rounds):
            start_idx = j * (8 * num_wires - 2)

            # Squeezing gates
            for i in range(num_wires):
                # for each wire, the number of parameters are i*2
                idx = start_idx + i * 2
                if idx + 1 < num_features:
                    qml.Squeezing(x[idx], x[idx + 1], wires=i)

            # Beamsplitter gates
            for i in range(num_wires - 1):
                # start_index + Squeezing gates, and then i*2 parameters for each gate
                idx = start_idx + num_wires * 2 + i * 2
                if idx + 1 < num_features:
                    qml.Beamsplitter(x[idx], x[idx + 1], wires=[i % num_wires, (i + 1) % num_wires])

            # Rotation gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters, and then i parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + i
                if idx < num_features:
                    qml.Rotation(x[idx], wires=i)

            # Displacement gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters + Rotation gates, and then i*2 parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + num_wires + i * 2
                if idx + 1 < num_features:
                    qml.Displacement(x[idx], x[idx + 1], wires=i)

            # Kerr gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters + Rotation gates + Displacement gates, and then i parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + i
                if idx < num_features:
                    qml.Kerr(x[idx], wires=i)

## **3. QNN Model**
To build a model, we need to
* define a layer
* build a circuit with the defined layer
* build a model with the defined circuit.

### **3.1 QNN Layer**
This in a faithful implementation of classical neural networks:
* Weight matrix: Interferometer 1 + Squeezing + Interferometer 2
* Bias addition: Displacement gate
* Nonlinear activation function: Kerr gate

In [20]:
def apply(v, num_wires):
        """
        Applies the quantum neural network layer with the given parameters.

        Parameters:
        - v (list or array): List or array of parameters for the quantum gates.

        Returns:
        - None
        """
        num_params = len(v)

        # Interferometer 1
        for i in range(num_wires - 1):
            idx = i * 2
            if idx + 1 < num_params:
                theta = v[idx]
                phi = v[idx + 1]
                qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + i
            if idx < num_params:
                qml.Rotation(v[idx], wires=i)

        # Squeezers
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + i
            if idx < num_params:
                qml.Squeezing(v[idx], 0.0, wires=i)

        # Interferometer 2
        for i in range(num_wires - 1):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + i * 2
            if idx + 1 < num_params:
                theta = v[idx]
                phi = v[idx + 1]
                qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + i
            if idx < num_params:
                qml.Rotation(v[idx], wires=i)

        # Bias addition
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + i
            if idx < num_params:
                qml.Displacement(v[idx], 0.0, wires=i)

        # Non-linear activation function
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + num_wires + i
            if idx < num_params:
                qml.Kerr(v[idx], wires=i)


**Weight Initializer**

Randomly initialized values are used as initial parameters of the QNN circuit.

In [33]:
def init_weights(layers, num_wires, active_sd=0.0001, passive_sd=0.1):
    """
    This is a weight vector initializer.
    Input: number of layers, number of wires
    Output: concatenated weight vector
    """
    M = (num_wires - 1) * 2 + num_wires  # Number of interferometer parameters

    int1_weights = np.random.normal(size=[layers, M], scale=passive_sd) #beamsplitters and rotations
    print("size for int1", int1_weights.size)
    s_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #squeezers
    print("size for s_weights", int1_weights.size)
    int2_weights = np.random.normal(size=[layers, M], scale=passive_sd) #beamsplitters and rotations
    print("size for int2", int2_weights.size)
    dr_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #displacement
    print("size for dr_weights", dr_weights.size)
    k_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #Kerr
    print("size for k_weights", k_weights.size)

    weights = np.concatenate(
        [int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)
    
    return weights

### **3.2 QNN Circuit**

For building a PA circuit as opposed to a qubit-based circuit, we have to choose "strawberryfields.fock" as device.

In [56]:
num_wires = 4
num_basis = 2
encoding = True
# select a device
dev = qml.device("strawberryfields.fock", wires=num_wires, cutoff_dim=num_basis)

@qml.qnode(dev, interface="torch")
def quantum_nn(inputs, var):
    """
    This is a quantum circuit composed of a data encoding circuit and a QNN circuit.
    Input: classical data (inputs), quantum parameters (var)
    Output: quantum state, converted to classical data after measurement
    """
    global encoding
    if (encoding):
    # convert classical inputs into quantum states
        encode(inputs, num_wires)
        print("ENCODED inputs")
    # iterative quantum layers
    encoding = False
    print("length of var", len(var))
    print("shape of var", var.shape)
    for v in var:
        print("Looking at v", v)
        apply(v, num_wires)

    # measure the resulting state and return
    return qml.expval(qml.X(0))

### **3.3 Model building**

In [57]:
num_layers = 2

def get_model(num_wires, num_layers):
    """
    This is a model building function.
    Input: number of modes, number of layers
    Output: PyTorch model
    """
    weights = init_weights(num_layers, num_wires)
    print("shape of weights", weights.shape)
    print("weights length: ", len(weights))
    print("weights looks like: ", weights[0][1])

    shape_tup = weights.shape
    weight_shapes = {'var': shape_tup}
    qlayer = qml.qnn.TorchLayer(quantum_nn, weight_shapes)
    model = torch.nn.Sequential(qlayer)
    return model

In [58]:
model = get_model(num_wires, num_layers)
print(model.parameters())
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of model parameters: {total_params}")
for p in model.parameters():
    print("parameter: ", p)

size for int1 20
size for s_weights 20
size for int2 20
size for dr_weights 8
size for k_weights 8
shape of weights (2, 32)
weights length:  2
weights looks like:  0.09846326275347056
<generator object Module.parameters at 0x17af112a0>
Sequential(
  (0): <Quantum Torch Layer: func=quantum_nn>
)
Total number of model parameters: 64
parameter:  Parameter containing:
tensor([[5.6802, 3.3705, 0.1740, 0.1997, 0.6523, 6.0451, 2.6568, 4.5011, 4.9300,
         4.0147, 0.3425, 2.8640, 4.3010, 5.1482, 1.0441, 2.6092, 1.9265, 0.0997,
         4.4783, 3.0309, 5.1186, 4.1797, 4.4110, 5.6194, 3.0499, 5.9269, 5.8403,
         0.7307, 5.7687, 3.4351, 4.4271, 3.3140],
        [6.0813, 0.6393, 2.5074, 1.5122, 5.6054, 3.3124, 0.8831, 0.9281, 1.8431,
         0.5053, 1.8957, 3.4127, 4.2681, 0.8699, 3.5787, 4.7822, 3.0957, 3.7877,
         1.9716, 6.0924, 3.8233, 3.8633, 3.8259, 4.1648, 4.8795, 1.4933, 3.2557,
         5.1905, 1.2950, 4.6301, 3.0443, 3.1046]], requires_grad=True)


## **4. Model Training**

In [60]:
# TRAINING MODEL (without batching)
loss_fn = torch.nn.L1Loss()  # binary cross-entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

n_epochs = 3

for epoch in range(n_epochs):
    print("====================================================================================")
    print(f"+++++++++++++++ Epoch {epoch} ++++++++++++++++++++++++++++++++++")
    print("====================================================================================")

    y_pred = model(X).reshape(-1, 1)  
    print(f"Size of y_pred: {y_pred.size()}, Values: {y_pred}")

    loss = loss_fn(y_pred, y)
    print(f"loss = {loss}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f'Finished epoch {epoch}, latest loss {loss}')

# #TRAINING MODEL
# loss_fn = nn.BCELoss()  # binary cross entropy
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# n_epochs = 1
# batch_size = 2
 
# for epoch in range(n_epochs):
#     for i in range(0, len(X), batch_size):
#         print("====================================================================================")
#         print(f"+++++++++++++++Batch number: {i} in Epoch {epoch}++++++++++++++++++++++++++++++++++")
#         print("====================================================================================")

#         Xbatch = X[i:i+batch_size]
#         print(f"Size of Xbatch: {Xbatch.size()}")
#         y_pred = model(Xbatch).reshape(-1, 1)  
#         print(f"Size of y_pred: {y_pred.size()}, Values: {y_pred}")

#         ybatch = y[i:i+batch_size]
#         loss = loss_fn(y_pred, ybatch)
#         print(f"loss = {loss}")
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     print(f'Finished epoch {epoch}, latest loss {loss}')


+++++++++++++++ Epoch 0 ++++++++++++++++++++++++++++++++++
length of var 2
shape of var torch.Size([2, 32])
Looking at v tensor([5.6802, 3.3705, 0.1740, 0.1997, 0.6523, 6.0451, 2.6568, 4.5011, 4.9300,
        4.0147, 0.3425, 2.8640, 4.3010, 5.1482, 1.0441, 2.6092, 1.9265, 0.0997,
        4.4783, 3.0309, 5.1186, 4.1797, 4.4110, 5.6194, 3.0499, 5.9269, 5.8403,
        0.7307, 5.7687, 3.4351, 4.4271, 3.3140], grad_fn=<UnbindBackward0>)
Looking at v tensor([6.0813, 0.6393, 2.5074, 1.5122, 5.6054, 3.3124, 0.8831, 0.9281, 1.8431,
        0.5053, 1.8957, 3.4127, 4.2681, 0.8699, 3.5787, 4.7822, 3.0957, 3.7877,
        1.9716, 6.0924, 3.8233, 3.8633, 3.8259, 4.1648, 4.8795, 1.4933, 3.2557,
        5.1905, 1.2950, 4.6301, 3.0443, 3.1046], grad_fn=<UnbindBackward0>)
length of var 2
shape of var torch.Size([2, 32])
Looking at v tensor([5.6802, 3.3705, 0.1740, 0.1997, 0.6523, 6.0451, 2.6568, 4.5011, 4.9300,
        4.0147, 0.3425, 2.8640, 4.3010, 5.1482, 1.0441, 2.6092, 1.9265, 0.0997,
        4.47

Finished epoch 2, latest loss 0.5


## **5. Evaluation**

In [ ]:
# EVALUATING
with torch.no_grad():
    y_pred = model(X)

    print(f"Size of y_pred (evaluation): {y_pred.size()}, Values: {y_pred}")

accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy: {accuracy}")

## **ISOLATED TESTING**

Beware and steer clear

In [93]:
# Encode the input data once outside the training loop
encoded_data = None
encoding = True
def encode_data_once(x, num_wires):
    """ Encodes the data only once and stores it. """
    global encoded_data
    if encoded_data is None:
        encode(x, num_wires)
        encoded_data = True  # Indicates that encoding has been done
        print(f"ENCODED inputs: {x}")
    else:
        print("Using cached encoded inputs.")

@qml.qnode(dev, interface="torch")
def quantum_nn(inputs, var):
    """ Quantum Neural Network with a caching mechanism for encoded inputs. """
    global encoding
    if encoding:
        encode_data_once(inputs, num_wires)  # Encode only once
        encoding = False
    print("length of var", len(var))
    print("shape of var", var.shape)
    for v in var:
        print("--- Applying Layer ---")
        print(f"Parameters for this layer: {v}")
        apply(v, num_wires)
    return qml.expval(qml.X(0))

In [94]:
# Simple test input
test_input = torch.tensor([0, 137, 40, 35, 168, 43.1, 2.288, 33], dtype=torch.float32)

# Initialize weights with known values or use random values for testing
test_weights = init_weights(num_layers, num_wires, active_sd=0.01, passive_sd=0.1)
test_weights = torch.tensor(test_weights, dtype=torch.float32)

# Define a simple model with the quantum layer
model = get_model(num_wires, num_layers)

# Set up the loss function and optimizer
loss_fn = nn.L1Loss()  # Mean Absolute Error Loss
optimizer = optim.SGD(model.parameters(), lr=0.5)

# Target output for the test case (arbitrary for testing purposes)
y_test = torch.tensor([1.0], dtype=torch.float32)

# Run the training loop for 5 iterations
n_epochs = 5

for epoch in range(n_epochs):
    print("====================================================================================")
    print(f"+++++++++++++++ Epoch {epoch} ++++++++++++++++++++++++++++++++++")
    print("====================================================================================")

    # Forward pass: predict using the model
    y_pred = model(test_input).reshape(-1, 1)
    print(f"Output from the quantum circuit with test input: {y_pred.item()}")

    # Compute loss
    loss = loss_fn(y_pred, y_test)
    print(f"Loss = {loss.item()}")

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Finished epoch {epoch}, latest loss {loss.item()}')


size for int1 20
size for s_weights 20
size for int2 20
size for dr_weights 8
size for k_weights 8
size for int1 20
size for s_weights 20
size for int2 20
size for dr_weights 8
size for k_weights 8
shape of weights (2, 32)
weights length:  2
weights looks like:  0.03538308315641023
+++++++++++++++ Epoch 0 ++++++++++++++++++++++++++++++++++
encoding rounds 1
ENCODED inputs: tensor([  0.0000, 137.0000,  40.0000,  35.0000, 168.0000,  43.1000,   2.2880,
         33.0000])
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
Parameters for this layer: tensor([1.0603, 2.8039, 2.7515, 5.1245, 1.2353, 1.3466, 3.7217, 1.2563, 2.6239,
        4.6288, 2.7507, 2.9677, 0.8495, 0.1244, 2.0035, 5.2682, 5.6754, 3.9982,
        1.4356, 5.8710, 3.0230, 2.0916, 1.0574, 2.2542, 1.5702, 5.1163, 2.9667,
        3.1096, 1.9404, 4.3269, 1.5989, 0.1363], grad_fn=<UnbindBackward0>)
--- Applying Layer ---
Parameters for this layer: tensor([0.2229, 1.3538, 3.8192, 2.4560, 0.2555, 4.6133, 3.0476